## Checking results between exposure record and metadata.

In [ ]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from lsst.obs.lsst import LsstCam
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from lsst.summit.utils.efdUtils import makeEfdClient
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])
instrument = 'LSSTCam'
camera = LsstCam.getCamera()
client = makeEfdClient()

In [ ]:
expId = 2025080600250
rawExp = butler.get('raw', detector=94, exposure=expId, instrument=instrument)
md = rawExp.getMetadata()
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)
(mountErrors, mountData) = calculateMountErrors(expRecord, client)

In [ ]:
# expRecord matches start values
print(md['RASTART'], expRecord.tracking_ra, (md['RASTART'] - expRecord.tracking_ra)*3600)
print(md['DECSTART'], expRecord.tracking_dec, (md['DECSTART'] - expRecord.tracking_dec)*3600)

In [ ]:
print(md['AZSTART'], md['AZEND'], expRecord.azimuth)
print(np.median(mountData.azimuthData['actualPosition'].values))
azs = mountData.azimuthData['actualPosition'].values
print((azs[0] + azs[-1]) / 2.0)
print(np.median(azs))

In [ ]:
print(md['ELSTART'], md['ELEND'], 90.0 - expRecord.zenith_angle)

In [ ]:
# Az/El move hundreds of arcseconds during the exposure
print((md['AZSTART'] - md['AZEND']) * 3600.0, (md['ELSTART'] - md['ELEND']) * 3600.0)

In [ ]:
# Times match well
print(Time(md['MJD-BEG'], scale='tai', format='mjd').utc.isot, expRecord.timespan.begin.utc.isot)
print(Time(md['MJD-END'], scale='tai', format='mjd').utc.isot, expRecord.timespan.end.utc.isot)